In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
from scipy.stats import linregress
from datetime import datetime

%matplotlib inline

# Impor the OpenWeatherMap API key
from config import api_key

In [ ]:
#Skip running this cell if your API limit is reached
#Variables used in API call, oldest date is furthest back we are willing to go
oldest_date = "2014-01-31"
company_ticker = ["TAN", "ENPH", "SEDG", "FSLR", "HYDR", "PLUG", "BLDP", "BE", "FAN", "TAC", "TPIC", "AMSC"]
time_function = "TIME_SERIES_MONTHLY"
base_url = "https://www.alphavantage.co/query?"

company_data = []

for ticker in company_ticker:
    finished_url = f"{base_url}function={time_function}&symbol={ticker}&apikey={api_key}"
    print(f"Processing {ticker}")

    result_data = requests.get(finished_url).json()

    time.sleep(1)

    for monthly in result_data["Monthly Time Series"]:
        monthly_date = time.strptime(monthly, "%Y-%m-%d")
        oldest = time.strptime(oldest_date, "%Y-%m-%d")
        if (monthly_date < oldest):
            continue
        stock_date = monthly
        stock_close = result_data["Monthly Time Series"][monthly]["4. close"]
        company_data.append({"Ticker":ticker,
                            "Date":stock_date,
                            "Closing Price":stock_close})

company_data_df = pd.DataFrame(company_data)
company_data_df['Date'] = pd.to_datetime(company_data_df['Date'])
company_data_df


In [ ]:
#Skip running this cell if your API limit is reached
#Add a type column and lable to each stock
company_data_df["Type"] = ""
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "TAN", "Solar", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "ENPH", "Solar", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "SEDG", "Solar", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "FSLR", "Solar", company_data_df["Type"])

company_data_df["Type"] = np.where(company_data_df["Ticker"] == "HYDR", "Hydrogen", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "PLUG", "Hydrogen", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "BLDP", "Hydrogen", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "BE", "Hydrogen", company_data_df["Type"])

company_data_df["Type"] = np.where(company_data_df["Ticker"] == "FAN", "Wind", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "TAC", "Wind", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "TPIC", "Wind", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Ticker"] == "AMSC", "Wind", company_data_df["Type"])

#Save the Data pulled to a CSV file
company_data_df.to_csv("data/company_data.csv", index_label="Stock_ID") #change stock id 

In [ ]:
#prep some lists for the later code to use
company_ticker = ["TAN", "ENPH", "SEDG", "FSLR", "HYDR", "PLUG", "BLDP", "BE", "FAN", "TAC", "TPIC", "AMSC"]
solar_tickers = ["TAN", "ENPH", "SEDG", "FSLR"]
hydrogen_tickers = ["BLDP", "PLUG", "BE", "HYDR"]
wind_tickers = ["FAN", "TAC", "TPIC", "AMSC"]
year_list = [i for i in range(2014, 2025)]

#Read data from the saved CSV file and sort it by Ticker and date
read_data_df = pd.read_csv("data/company_data.csv", index_col="Stock_ID") #change stock id
read_data_df['Date'] = pd.to_datetime(read_data_df['Date'])
ordered_data_df = read_data_df.sort_values(by=["Ticker", "Date"])
ordered_data_df

In [3]:
#Function to plot graphs showing stock prices
def plot_stock_prices(tickers,given_data_df,type):
    plt.figure(figsize=(8, 6))

    for ticker in tickers:
        stock_data = given_data_df[given_data_df["Ticker"] == ticker]
        plt.plot(stock_data["Date"], stock_data["Closing Price"], label=ticker)

    plt.title(f"Stock Prices for {type} Companies")
    plt.xlabel("Date")
    plt.ylabel("Closing Price")
    
    plt.gca().xaxis.set_major_locator(mdates.YearLocator())  # Major ticks every year
    plt.gca().xaxis.set_minor_locator(mdates.MonthLocator())  # Minor ticks every month
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format as year
    
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"graphs/{type}-stocks.png")
    plt.show()

In [4]:
#Function to plot graphs showing average yearly stock prices
def plot_avg_prices(tickers,given_data_df,type):
    plt.figure(figsize=(8, 6))

    for ticker in tickers:
        stock_data = given_data_df[given_data_df["Ticker"] == ticker]
        plt.plot(stock_data["Year"], stock_data["Avg. Closing Price"], label=ticker)

    plt.title(f"Avg. Yealy Stock Prices for {type} Companies")
    plt.xlabel("Year")
    plt.ylabel("Avg. Closing Price")
    plt.xticks(ticks=year_list, labels=[str(i) for i in year_list], rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"graphs/{type}-avg-yearly-stocks.png")
    plt.show()

In [5]:
#Function to plot graphs showing max yearly stock prices
def plot_max_prices(tickers,given_data_df,type):
    plt.figure(figsize=(8, 6))

    for ticker in tickers:
        stock_data = given_data_df[given_data_df["Ticker"] == ticker]
        plt.plot(stock_data["Year"], stock_data["Max Closing Price"], label=ticker)

    plt.title(f"Max Yealy Stock Prices for {type} Companies")
    plt.xlabel("Year")
    plt.ylabel("Max Closing Price")
    plt.xticks(ticks=year_list, labels=[str(i) for i in year_list], rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"graphs/{type}-max-yearly-stocks.png")
    plt.show()

In [6]:
#Function to plot graphs showing average yearly stock prices by company Sector
def plot_avg_type_prices(types,given_data_df):
    plt.figure(figsize=(8, 6))

    for type in types:
        stock_data = given_data_df[given_data_df["Type"] == type]
        plt.plot(stock_data["Year"], stock_data["Avg. Closing Price"], label=type)

    plt.title(f"Avg. Yealy Stock Prices by Sector")
    plt.xlabel("Year")
    plt.ylabel("Avg. Closing Price")
    plt.xticks(ticks=year_list, labels=[str(i) for i in year_list], rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"graphs/sector-avg-yearly-stocks.png")
    plt.show()

In [7]:
#Function to plot graphs showing max yearly stock prices by company Sector
def plot_max_type_prices(types,given_data_df):
    plt.figure(figsize=(8, 6))

    for type in types:
        stock_data = given_data_df[given_data_df["Type"] == type]
        plt.plot(stock_data["Year"], stock_data["Max Closing Price"], label=type)

    plt.title(f"Max Yealy Stock Prices by Sector")
    plt.xlabel("Year")
    plt.ylabel("Avg. Closing Price")
    plt.xticks(ticks=year_list, labels=[str(i) for i in year_list], rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"graphs/sector-max-yearly-stocks.png")
    plt.show()

In [ ]:
# Plot Stocks for each company
plot_stock_prices(solar_tickers,ordered_data_df,"Solar")
plot_stock_prices(hydrogen_tickers,ordered_data_df,"Hydrogen")
plot_stock_prices(wind_tickers,ordered_data_df,"Wind")

In [ ]:
#Yearly Average stock price for each company
yearly_average = []
for ticker in company_ticker:
    starting_year = ordered_data_df[ordered_data_df["Ticker"] == ticker]["Date"].min().strftime('%Y')
    for year in range(int(starting_year), 2025):
        years = ordered_data_df["Date"].dt.strftime('%Y')
        average = ordered_data_df.loc[(ordered_data_df["Ticker"] == ticker) & (years == str(year)), "Closing Price"].mean()
        yearly_average.append({"Ticker":ticker,
                                "Year":year,
                                "Avg. Closing Price":average,
                                "Type":ordered_data_df.loc[ordered_data_df["Ticker"] == ticker,"Type"].iloc[0]})

yearly_average_df = pd.DataFrame(yearly_average)
yearly_average_df  

In [ ]:
#Yearly max stock price for each company
yearly_max = []
for ticker in company_ticker:
    starting_year = ordered_data_df[ordered_data_df["Ticker"] == ticker]["Date"].min().strftime('%Y')
    for year in range(int(starting_year), 2025):
        years = ordered_data_df["Date"].dt.strftime('%Y')
        max = ordered_data_df.loc[(ordered_data_df["Ticker"] == ticker) & (years == str(year)), "Closing Price"].max()
        yearly_max.append({"Ticker":ticker,
                                "Year":year,
                                "Max Closing Price":max,
                                "Type":ordered_data_df.loc[ordered_data_df["Ticker"] == ticker,"Type"].iloc[0]})

yearly_max_df = pd.DataFrame(yearly_max)
yearly_max_df

In [ ]:
#Plot Avg. Yearly Stock Prices for each company
plot_avg_prices(solar_tickers,yearly_average_df,"Solar")
plot_avg_prices(hydrogen_tickers,yearly_average_df,"Hydrogen")
plot_avg_prices(wind_tickers,yearly_average_df,"Wind")

In [ ]:
#Plot Max Yearly Stock Prices for each company
plot_max_prices(solar_tickers,yearly_max_df,"Solar")
plot_max_prices(hydrogen_tickers,yearly_max_df,"Hydrogen")
plot_max_prices(wind_tickers,yearly_max_df,"Wind")

In [ ]:
#Yearly Average stock price for each sector
yearly_average_sector = []
for types in ordered_data_df["Type"].unique():
    starting_year = ordered_data_df[ordered_data_df["Type"] == types]["Date"].min().strftime('%Y')
    for year in range(int(starting_year), 2025):
        years = ordered_data_df["Date"].dt.strftime('%Y')
        average = ordered_data_df.loc[(ordered_data_df["Type"] == types) & (years == str(year)), "Closing Price"].mean()
        yearly_average_sector.append({"Type":types,
                                "Year":year,
                                "Avg. Closing Price":average,})

yearly_avg_sector_df = pd.DataFrame(yearly_average_sector)
yearly_avg_sector_df

In [ ]:
#Yearly Max stock price for each sector
yearly_max_sector = []
for types in ordered_data_df["Type"].unique():
    starting_year = ordered_data_df[ordered_data_df["Type"] == types]["Date"].min().strftime('%Y')
    for year in range(int(starting_year), 2025):
        years = ordered_data_df["Date"].dt.strftime('%Y')
        max = ordered_data_df.loc[(ordered_data_df["Type"] == types) & (years == str(year)), "Closing Price"].max()
        yearly_max_sector.append({"Type":types,
                                "Year":year,
                                "Max Closing Price":max,})

yearly_max_sector_df = pd.DataFrame(yearly_max_sector)
yearly_max_sector_df

In [ ]:
#Plot Avg and Max yearly stock prices by sector
plot_avg_type_prices(["Wind", "Hydrogen", "Solar"],yearly_avg_sector_df)
plot_max_type_prices(["Wind", "Hydrogen", "Solar"],yearly_max_sector_df)